# Homework lesson2 Scientists

There are two whole numbers:
1 < a,b < 100

One scientist("Sum") get provided with sum of numbers,
another  ("Prod") get provided with product of numbers. 
Both scientists know that numbers 1 < a,b <100.
                                        
Determine the numbers being based on the following dialog: 
    Prod: I don't know the numbers;
    Sum: I know it;
    Prod: then I know the numbers; 
    Sum: then I know the numbers too.

#### Ініціалізація

In [13]:
import numpy as np
SUM_RESULTS = np.arange(4, 199, dtype=np.int32) # всі можливі суми чисел

# Допоміжні функції:

#### Ця функція приймає число, та повертає масив всіх комбінацій добутків, результатом обчислення яких, буде число надане функції

In [14]:
def factor_pairs(number):
    '''
    Input: a number which has to be factorized
    Output: list of factor pairs of the product that is equal to the input number

    Example:
    Input: 20
    Output: [[10, 2], [4, 5]]
    '''
    factors = []
    for i in range(2, int(number**0.5) + 1):
        if number % i == 0:
            factors.append([i, number // i])
    return factors

#### Ця функція приймає число, та повертає True або False, в залежності від того чи є число простим.

In [15]:
def is_prime(number):
    '''
    Input: a number which has to be checked if it is prime
    Output: True if the number is prime, False otherwise

    Prime numbers: 2, 3, 5, 7, 11, 13, 17, 19, 23, 29...
    '''
    if number <= 1:
        return False
    elif number <= 3:
        return True
    elif number % 2 == 0 or number % 3 == 0:
        return False
    i = 5
    while i * i <= number:
        if number % i == 0 or number % (i + 2) == 0:
            return False
        i += 6
    return True

#### Ця функція приймає число, та повертає масив комбінацій чисел, сума яких дорівнює числу, яке було надане функції

In [16]:
def split_into_sums(number):
    '''
    Input: a number which has to be split into sums
    Output: list of all possible combinations of two numbers that sum up to the input number

    Example:
    Input: 8
    Output: [[2,6],[3,5],[4,4]]
    '''
    # split combinations into sums
    sum_combinations = np.array([], dtype=np.int32).reshape(0, 2)
    for i in range(2, number // 2 + 1):
        sum_combinations = np.append(sum_combinations, [(i, number - i)], axis=0)
    return sum_combinations

# Рішення:

## 1. Добуток: "Я не знаю ці числа"

Це означає, що добуток точно не є унікальним, тобто таким, який утворює тільки якась одна пара чисел.

Наприклад якщо добуток - 21, то він є ункіальним, оскільки існує лише одна пара чисел, яка його утворює (3, 7)

Якби добуток був унікальним, то можна було б зразу здогадатись що це за числа.

Також з цього можна зробити висновок, що обидва числа не можуть бути одночасно простими, оскільки добуток двох простих чисел є завжди унікальним.

In [17]:
def first_move(sum_results):
    '''
    Input: list of all possible sum combinations
    Output: list of all possible sum combinations after the first phrase
    (removing all sum combinations where both numbers are prime)
    '''
    for sum_result in sum_results:
        possible_numbers = split_into_sums(sum_result)
        for possible_number in possible_numbers:
            if is_prime(possible_number[0]) and is_prime(possible_number[1]):
                sum_results = np.delete(sum_results, np.where(sum_results == sum_result), axis=0)
                break
    return sum_results

## 2. Сума: "Я знаю, що ти не знаєш"

З цих слів можна зробити висновок, що число, яке сказали цьому вченому точно не є сумою двох простих чисел, або інших чисел, добуток яких є унікальним.

Цей вчений побачив, що зі всіх можливих пар чисел, які можуть складати дану йому суму, не можна утворити унікальний добуток. Відповідно він знав, що вчений, якому сказали добуток, не назве правильну пару чисел, оскільки добуток, який йому назвали не є унікальним і його можуть утворити більше ніж одна пара чисел.

#### Ця функція поверне масив, в якому комбінації чисел розбиті по сумах, наприклад [[[2, 9], [3, 8], [4, 7]], _[[4, 13]]_, [[4, 19], [7, 16]]]

In [18]:
def second_move(sum_results):
    '''
    Input: list of all possible sum combinations
    Output: list of all possible answers that deal with the second phrase(rules)
    Description:
    1. Loop over all possible sum combinations
    2. Split each sum combination into combinations of two numbers that sum up to the sum combination
    3. Get all factor pairs of the product of two numbers
    4. For each combination of factor pairs check if it's number are not prime(if yes, then skip this combination)
    5. If the sum of the factor pairs is in the list of all possible sum combinations, then it's not the answer
    6. Append the combination of two numbers to the list of possible answers

    '''
    possible_answers = []
    for i, sum_result in enumerate(sum_results):
        possible_numbers = split_into_sums(sum_result)
        possible_answers.append([]) # масив для збереження комбінацій суми 
        for possible_number in possible_numbers:
            factor_pairs_list = factor_pairs(possible_number[0] * possible_number[1])
            append = True
            for factor_pair in factor_pairs_list:
                if is_prime(factor_pair[0]) and is_prime(factor_pair[1]):
                    append = False
                    break
                if sum(factor_pair) in sum_results:
                    if factor_pair == possible_number.tolist():
                        continue
                    append = False
                    break
            if append:
                possible_answers[i].append(possible_number.tolist()) # додає комбінацію до масиву сум
    return possible_answers

## 3. Добуток: "Тепер я знаю ці числа" та Сума: "Тепер я також знаю ці числа"

Тепер для того, щоб визначити правильну комбінацію чисел, нам потрібно знайти суму, якій відповідає тільки один добуток.

In [19]:
def third_move(possible_answers):
    '''
    Input: list of all possible answers that we got after getting the rules from a dialogue
    Output: the combination(chooses the answer where is only one possible combination in the sum)
    '''
    # select the answer where is only one possible combination
    for possible_answer in possible_answers:
        if len(possible_answer) == 1:
            return possible_answer

## Запуск програми

In [20]:
from datetime import datetime
startTime = datetime.now()
print(f'Початкова кількість сум, які можуть бути відповіддю: {len(SUM_RESULTS)}')

SUM_RESULTS = first_move(SUM_RESULTS)
print(f'Кількість сум, які можуть бути відповіддю після фрази : Добуток: "Я не знаю ці числа": {len(SUM_RESULTS)}')

possible_answers = second_move(SUM_RESULTS)
print(f'Кількість комбінацій чисел, які можуть бути відповіддю після фрази : Сума: "Я знаю, що ти не знаєш": {len(possible_answers)}')

answer = third_move(possible_answers)
print(f'Відповідь після фраз Добуток: "Тепер я знаю ці числа" та Сума: "Тепер я також знаю ці числа": {answer}')
print(f'Час виконання: {datetime.now() - startTime}')

Початкова кількість сум, які можуть бути відповіддю: 195
Кількість сум, які можуть бути відповіддю після фрази : Добуток: "Я не знаю ці числа": 54
Кількість комбінацій чисел, які можуть бути відповіддю після фрази : Сума: "Я знаю, що ти не знаєш": 54
Відповідь після фраз Добуток: "Тепер я знаю ці числа" та Сума: "Тепер я також знаю ці числа": [[4, 13]]
Час виконання: 0:00:00.391320
